In [2]:
import pandas as pd
import numpy as np



In [4]:
from google.colab import files
uploaded = files.upload()

Saving CrimesOnWomenData.csv to CrimesOnWomenData.csv


In [5]:
df = pd.read_csv("CrimesOnWomenData.csv")
df.head()

,Unnamed: 0,State,Year,Rape,K&A,DD,AoW,AoM,DV,WT
0,0,ANDHRA PRADESH,2001,871,765,420,3544,2271,5791,7
1,1,ARUNACHAL PRADESH,2001,33,55,0,78,3,11,0
2,2,ASSAM,2001,817,1070,59,850,4,1248,0
3,3,BIHAR,2001,888,518,859,562,21,1558,83
4,4,CHHATTISGARH,2001,959,171,70,1763,161,840,0


workig on dataset

1- calculating total crime per row

In [6]:
crime_columns = ['Rape', 'K&A', 'DD', 'AoW', 'AoM', 'DV', 'WT']
df["Total_Crimes"] = df[crime_columns].sum(axis=1)


2- normalising in range 0-5

In [7]:
max_crime = df["Total_Crimes"].max()
df["Normalized_Crime_Factor"] = (df["Total_Crimes"] / max_crime) * 5


3- hour-wise crime likelihood distribution

In [8]:
hourly_distribution = {
    0: 0.06, 1: 0.04, 2: 0.03, 3: 0.02, 4: 0.01,
    5: 0.01, 6: 0.01, 7: 0.01, 8: 0.02, 9: 0.02,
    10: 0.02, 11: 0.03, 12: 0.04, 13: 0.04, 14: 0.04,
    15: 0.05, 16: 0.06, 17: 0.07, 18: 0.08, 19: 0.08,
    20: 0.07, 21: 0.07, 22: 0.06, 23: 0.06
}

4-generate Synthetic Data (100 Samples per Row)

In [10]:


modes = ["walking", "cab", "metro"]
mode_penalty_map = {"walking": 2, "cab": 1, "metro": 0.5}
synthetic_rows = []

for _, row in df.iterrows():
    for _ in range(100):
        hour = np.random.choice(list(hourly_distribution.keys()), p=list(hourly_distribution.values()))
        battery = np.random.randint(5, 101)
        is_alone = np.random.choice([0, 1])
        mode = np.random.choice(modes)

        # Time-based risk
        if 0 <= hour <= 4:
            time_risk = 1.8
        elif 5 <= hour <= 7:
            time_risk = 0.3
        elif 8 <= hour <= 17:
            time_risk = 0.5
        elif 18 <= hour <= 20:
            time_risk = 1.0
        elif 21 <= hour <= 23:
            time_risk = 1.5

        alone_penalty = 1 if is_alone else 0
        battery_penalty = 1 if battery < 20 else 0
        mode_penalty = mode_penalty_map[mode]
        noise = np.random.uniform(-0.2, 0.2)

        risk_score = row["Normalized_Crime_Factor"] + time_risk + alone_penalty + battery_penalty + mode_penalty + noise
        risk_score = round(min(10, max(1, risk_score)), 1)

        synthetic_rows.append({
            "State": row["State"],
            "Year": row["Year"],
            "Total_Crimes": row["Total_Crimes"],
            "Normalized_Crime_Factor": round(row["Normalized_Crime_Factor"], 2),
            "hour": hour,
            "battery_percent": battery,
            "is_alone": is_alone,
            "mode_of_travel": mode,
            "risk_score": risk_score
        })


In [11]:
synthetic_df = pd.DataFrame(synthetic_rows)
synthetic_df.head()

,State,Year,Total_Crimes,Normalized_Crime_Factor,hour,battery_percent,is_alone,mode_of_travel,risk_score
0,ANDHRA PRADESH,2001,13669,1.31,18,80,1,cab,4.3
1,ANDHRA PRADESH,2001,13669,1.31,7,73,1,cab,3.8
2,ANDHRA PRADESH,2001,13669,1.31,14,60,0,walking,3.7
3,ANDHRA PRADESH,2001,13669,1.31,18,90,0,walking,4.3
4,ANDHRA PRADESH,2001,13669,1.31,15,29,0,walking,3.7


5- creating a synthtic dataset csv file

In [12]:
synthetic_df.to_csv("SyntheticSafetyData.csv", index=False)

from google.colab import files
files.download("SyntheticSafetyData.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

WORKING ON THIS DATASET USING RANDOM FOREST REGRESSION MODEL

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np


In [14]:
df = pd.read_csv("SyntheticSafetyData.csv")
df.head()

,State,Year,Total_Crimes,Normalized_Crime_Factor,hour,battery_percent,is_alone,mode_of_travel,risk_score
0,ANDHRA PRADESH,2001,13669,1.31,18,80,1,cab,4.3
1,ANDHRA PRADESH,2001,13669,1.31,7,73,1,cab,3.8
2,ANDHRA PRADESH,2001,13669,1.31,14,60,0,walking,3.7
3,ANDHRA PRADESH,2001,13669,1.31,18,90,0,walking,4.3
4,ANDHRA PRADESH,2001,13669,1.31,15,29,0,walking,3.7


Encode Categorical Column (mode_of_travel)

In [18]:
df_encoded = pd.get_dummies(df, columns=["mode_of_travel"])
df_encoded.head()

,State,Year,Total_Crimes,Normalized_Crime_Factor,hour,battery_percent,is_alone,risk_score,mode_of_travel_cab,mode_of_travel_metro,mode_of_travel_walking
0,ANDHRA PRADESH,2001,13669,1.31,18,80,1,4.3,True,False,False
1,ANDHRA PRADESH,2001,13669,1.31,7,73,1,3.8,True,False,False
2,ANDHRA PRADESH,2001,13669,1.31,14,60,0,3.7,False,False,True
3,ANDHRA PRADESH,2001,13669,1.31,18,90,0,4.3,False,False,True
4,ANDHRA PRADESH,2001,13669,1.31,15,29,0,3.7,False,False,True


target Encode State Based on Average risk_score

In [19]:
# average risk score for each state
state_risk_map = df_encoded.groupby("State")["risk_score"].mean().to_dict()

# mapping each row's 'State' to its encoded numeric risk
df_encoded["state_risk_encoded"] = df_encoded["State"].map(state_risk_map)
df_encoded = df_encoded.drop(columns=["State"])


In [20]:
X = df_encoded.drop(columns=["Year", "risk_score"])
y = df_encoded["risk_score"]

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [34]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", round(mse, 2))
print("R² Score:", round(r2, 2))

Mean Squared Error: 0.02
R² Score: 0.99
